In [ ]:
'''
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"# this is potentially unsafe but who cares cos it gets rid of an error!

%cd "C:/Users/Ibrahim/biomechanics/yolov7"

filesname= "CL_0.6_19"
files= str(filesname) + ".mp4" 



#!python detect_or_track.py --weights yolov7_custom.pt --source {files} --save-txt --device cpu --name {filesname}

'''

In [1]:
import os

path = 'C:/Users/c1606436/biomechanics/yolov7/runs/detect/'
folders = os.listdir(path)

for folder in folders:
    filesname = folder
    files = filesname + '.mp4'
    print(filesname)

CT_0.6_1
CT_0.6_10
CT_0.6_11
CT_0.6_12
CT_0.6_13
CT_0.6_14
CT_0.6_15
CT_0.6_16
CT_0.6_17
CT_0.6_18
CT_0.6_19
CT_0.6_2
CT_0.6_20
CT_0.6_3
CT_0.6_4
CT_0.6_5
CT_0.6_6
CT_0.6_7
CT_0.6_8
CT_0.6_9
CT_0.8_1
CT_0.8_10
CT_0.8_11
CT_0.8_12
CT_0.8_13
CT_0.8_14
CT_0.8_15
CT_0.8_16
CT_0.8_17
CT_0.8_18
CT_0.8_19
CT_0.8_2
CT_0.8_20
CT_0.8_3
CT_0.8_4
CT_0.8_5
CT_0.8_6
CT_0.8_7
CT_0.8_8
CT_0.8_9
CT_1.0_1
CT_1.0_10
CT_1.0_11
CT_1.0_12
CT_1.0_13
CT_1.0_14
CT_1.0_15
CT_1.0_16
CT_1.0_17
CT_1.0_18
CT_1.0_19
CT_1.0_2
CT_1.0_20
CT_1.0_3
CT_1.0_4
CT_1.0_5
CT_1.0_6
CT_1.0_7
CT_1.0_8
CT_1.0_9
WC_0.8_4
WC_1.0_16
WC_1.0_17


In [2]:
import cv2 

print(filesname)
filepath = "C:/Users/c1606436/biomechanics/yolov7/runs/detect/" + filesname
os.chdir(filepath)

def click_event(event, x, y, flags, params):
   if event == cv2.EVENT_LBUTTONDOWN:
    #print(f'Original Coordinates : ({x},{y})')
    global xy 
    global yy
    xy = x/frame.shape[1]
    yy = y/frame.shape[0]
    print(f'YOLO Coordinates : ({xy},{yy})')
    cv2.putText(frame, f'({x},{y})',(x,y),cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    
    cv2.circle(frame, (x,y), 8, (255,0,0), -1)
   
    return xy , yy
    
files= filesname +'.mp4'
videopath = files

videcap = cv2.VideoCapture(videopath)

cv2.namedWindow("1")
cv2.setMouseCallback("1",click_event)    
            
while(videcap.isOpened()):
    ret, frame = videcap.read()
    if ret == True:
        cv2.imshow('1', frame)   

        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
    else:
        break
cv2.destroyAllWindows()


WC_1.0_4
YOLO Coordinates : (0.07222222222222222,0.3453703703703704)


In [3]:
## biomechanics yolo
#!python detect_or_track.py --weights yolov7_custom.pt --source CL_1_S0003.mp4 --save-txt --name CL_1_S0003
import numpy as np
import os
import pandas as pd
import scipy
import math

print(filesname)
# Specify the folder containing the text files
folder_path = "C:/Users/c1606436/biomechanics/yolov7/runs/detect/" + filesname + "/labels"

# Create an empty list to store the data from each file
all_data = []

# Iterate through all the files in the folder
for filename in sorted(os.listdir(folder_path), key=lambda x: int(x.split("_")[-1].rstrip(".txt"))):
    if filename.endswith(".txt"):
        # Open the file and read its contents
        with open(os.path.join(folder_path, filename), 'r') as file:
            file_data = file.readline()

        # Split the line into a list of numbers
        line_data = file_data.strip().split(" ")

        # Convert the numbers to integers
        line_data = [float(x) for x in line_data]

        # Add the line data to the list of all data
        all_data.append(line_data)

# Convert the list of data to a pandas DataFrame
df = pd.DataFrame(all_data)
#display(df)

#df.to_csv('filename.csv', index=False)

# Write the DataFrame to a CSV file in the same folder as the text files
#df.to_csv(os.path.join(folder_path, "concatenated_data.csv"), index=False)


# Set the directory to the current directory #os.chdir("C:\\Users\\Ibrahim\\desktop")
file= videopath #for name currently


x_wall = xy #coordinates of the wall, uses the global variables from the other script
y_wall = yy

num_cont_frames=0
num_count_inbound = 0
#video = cv.VideoCapture(path)
ball_size = 0.22  #diameter of a regulation ball in meters
fps_cam = 10000  # Change this to the required fps of the video

# Initialize variable to track state of ball (in contact with wall or not)
in_contact = False

# Initialize variable to track whether in_contact has ever been True
in_contact_ever = False


# Initialize lists for inbound and outbound velocities
inbound_velocities = []
outbound_velocities = []

# Calculate time interval between frames in seconds
time_interval = 1 / fps_cam

scale  = []
x_def = []
inbound_x = []
inbound_y = []
outbound_x = []
outbound_y = []
threshold = 0.05 # adjust this value based on your data


for i, (index, row) in enumerate(df.iterrows()): #change this to for each line of the pandas df
#x is x center y is y center
    x, y, w, h = row[1], row[2], row[3], row[4]
    x2=x+w/2
    y2=y+h/2
    x1=x-w/2
    y1=y-h/2
    
    
    scale.append(ball_size/w)   #meters per pixel.diameter in pixels or coordinate value / real diameter in m to give pixel per m for a scale factor
    
    if x1 < x_wall: #sometimes the bbox is the wrong way around
        # Set in_contact to True
        in_contact = True
        # Set in_contact_ever to True
        in_contact_ever = True
        # Increment counter
        num_cont_frames = num_cont_frames + 1
        x_defe = x2-x_wall
        x_def.append(x_defe)
        
        
        
    else:
        in_contact = False

    if in_contact == False and in_contact_ever==False:
        num_count_inbound  = num_count_inbound  + 1

    if in_contact == False and in_contact_ever==True:
        outbound_x.append(x2) #list of x positions of right edge
        outbound_y.append(y2)
  
    if in_contact == True:
        if i<len(df)-1:
            next_row = df.iloc[i+1]
            next_x1 = next_row[1] - next_row[3]/2
            if abs(x1 - next_x1) > threshold:
                df.drop(index+1, inplace=True)
                continue
    
# Modifying the number of rows to take from the dataframe if num_count_inbound is less than 10
if num_count_inbound < 10:
    num_count_inbound += 1

# Taking the specified number of rows from the dataframe
result = df.head(num_count_inbound)

# Calculating x2 and y2 for each row in the result dataframe
for index, row in result.iterrows():
    x2 = row[1] + 0.5 * row[3]
    y2 = row[2] + 0.5 * row[4]
    
    inbound_x.append(x2)
    inbound_y.append(y2)
 
scale_ave=scipy.stats.trim_mean(scale, 0.3) #trim_mean 30% either way to remove some extraneous results

#display(df)
#df.to_csv('filename.csv', index=False)


contact_time = num_cont_frames/fps_cam

if x_def:
    realxdef = min(x_def)*scale_ave
else:
    realxdef = 0



inbound_x_diff = []
inbound_y_diff = []

inbound_x_velocities = []
inbound_y_velocities = []
inbound_len = len(inbound_x) - 1


# Calculate differences between consecutive x and y coordinates
for i in range(inbound_len):
    inbound_x_diff.append(inbound_x[i] - inbound_x[i + 1])
    inbound_y_diff.append(inbound_y[i] - inbound_y[i + 1])


filtered_inbound_x_diff = []
filtered_inbound_y_diff = []

for i in range(inbound_len):
    if inbound_x_diff[i] <= 1:
        filtered_inbound_x_diff.append(inbound_x_diff[i])
inbound_x_diff = filtered_inbound_x_diff


for i in range(inbound_len):
    if inbound_y_diff[i] <= 1:
        filtered_inbound_y_diff.append(inbound_y_diff[i])
inbound_y_diff = filtered_inbound_y_diff

inbound_y_diff = [abs(value) for value in inbound_y_diff]
inbound_x_diff = [abs(value) for value in inbound_x_diff]

# Calculate inbound velocities in meters per second
inbound_velocities = []
for i in range(inbound_len):
    inbound_x_velocity = inbound_x_diff[i] * scale_ave * fps_cam
    inbound_x_velocities.append(inbound_x_velocity)
    print(inbound_x_velocity)
    inbound_y_velocity = inbound_y_diff[i] * scale_ave * fps_cam
    inbound_y_velocities.append(inbound_y_velocity)

# Calculate outbound velocities
outbound_x_diff = []
outbound_y_diff = []

outbound_len = len(outbound_x) - 1

# Calculate differences between consecutive x and y coordinates
for i in range(outbound_len):
    outbound_x_diff.append(outbound_x[i] - outbound_x[i + 1])
    outbound_y_diff.append(outbound_y[i] - outbound_y[i + 1])
    
outbound_y_diff = [abs(value) for value in outbound_y_diff]
outbound_x_diff = [abs(value) for value in outbound_x_diff]


    
filtered_outbound_x_diff = []
filtered_outbound_y_diff = []

for i in range(outbound_len):
    if outbound_x_diff[i] <= 1:
        filtered_outbound_x_diff.append(outbound_x_diff[i])
outbound_x_diff = filtered_outbound_x_diff

for i in range(outbound_len):
    if outbound_y_diff[i] <= 1:
        filtered_outbound_y_diff.append(outbound_y_diff[i])
outbound_y_diff = filtered_outbound_y_diff


# Calculate outbound velocities in meters per second
outbound_velocities = []
outbound_x_velocities = []
outbound_y_velocities = []

for i in range(outbound_len):
    outbound_x_velocity = outbound_x_diff[i] * scale_ave * fps_cam
    outbound_x_velocities.append(outbound_x_velocity)
    outbound_y_velocity = outbound_y_diff[i] * scale_ave * fps_cam
    outbound_y_velocities.append(outbound_y_velocity)


  
    
filtered_outbound_x_velocities = []
for value in outbound_x_velocities:
    if value < 100:
        filtered_outbound_x_velocities.append(value)
outbound_x_velocities = filtered_outbound_x_velocities

filtered_outbound_y_velocities = []
for value in outbound_y_velocities:
    if value < 100:
        filtered_outbound_y_velocities.append(value)
outbound_y_velocities = filtered_outbound_y_velocities

filtered_inbound_x_velocities = []
for value in inbound_x_velocities:
    if value < 100:
        filtered_inbound_x_velocities.append(value)
inbound_x_velocities = filtered_inbound_x_velocities

filtered_inbound_y_velocities = []
for value in inbound_y_velocities:
    if value < 100:
        filtered_inbound_y_velocities.append(value)
inbound_y_velocities = filtered_inbound_y_velocities


corrected_average_inbound_x_velocities = scipy.stats.trim_mean(inbound_x_velocities, 0.2)
corrected_average_inbound_y_velocities = scipy.stats.trim_mean(inbound_y_velocities, 0.2)
corrected_average_inbound_velocities = math.hypot(corrected_average_inbound_y_velocities, corrected_average_inbound_x_velocities)
corrected_average_outbound_x_velocities = scipy.stats.trim_mean(outbound_x_velocities, 0.2)
corrected_average_outbound_y_velocities = scipy.stats.trim_mean(outbound_y_velocities, 0.2)
corrected_average_outbound_velocities = math.hypot(corrected_average_outbound_y_velocities, corrected_average_outbound_x_velocities)


# Create a dictionary with the data for the table
speeddata={'inbound x velocity ': corrected_average_inbound_x_velocities, 'inbound y velocity': corrected_average_inbound_y_velocities, 'inbound velocity': corrected_average_inbound_velocities, 'outbound x velocity': corrected_average_outbound_x_velocities, 'outbound y velocity': corrected_average_outbound_y_velocities, 'outbound velocity': corrected_average_outbound_velocities, 'contact time': contact_time, 'deformation' :realxdef}

df2 = pd.DataFrame(speeddata, index=range(len(speeddata)))

# Export the DataFrame to a CSV file
filename = file + 'results.csv'
df2.to_csv(filename, index=False)

del df
del df2




WC_1.0_4
32.72376835790381
21.818463578125407
32.72376835790424
18.182052981770884
4.359856124258405e-13
32.72376835790424
14.543678880775294
18.180089477129382
123.62617924819135
18.178125972487887
29.09128477083271
7.268894183425181
83.627626192937
32.72376835790381
21.816500073483038
61.81701663337889
10.905304779779271
3.636410596354089
7.270857688067116
7.270857688067116
43.634963651608885
25.45094716519563
32.727695367186804
7.268894183425181
29.091284770833585
14.545642385416793
29.08539425690822
7.270857688066244
7.274784697350113
21.818463578125407
32.72180485326187
32.72769536718768
21.814536568841543
10.909231789062268
32.72769536718768
10.905304779779271
14.545642385416356
10.907268284420772
18.182052981770884
43.631036642325014
18.182052981770884
39.99658955061243
7.272821192708178
29.091284770833585
10.905304779779271
21.818463578124536
10.905304779779271
25.45487417447906
36.36017895425833
14.545642385416356
36.36017895425833
7.272821192708615
36.36214245889983
7.2708576

In [4]:
import psutil
import os

files

# Find the process that is using the file
for process in psutil.process_iter():
    try:
        for item in process.open_files():
            if item.path == files:
                process.kill()
                break
    except (psutil.NoSuchProcess, psutil.AccessDenied, psutil.ZombieProcess):
        pass

# Remove the file
os.remove(filename)


In [ ]:
import shutil
src_path = "C:/Users/c1606436/biomechanics/yolov7/runs/detect/" + filesname 
src = src_path
dst = 'C:/Users/c1606436/biomechanics/yolov7/done'
shutil.move(src, dst)
